In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import pickle
sys.path.append(os.path.abspath(os.path.join('data')))
sys.path.insert(0,'../scripts/')
import warnings
warnings.filterwarnings('ignore')

from util import Util
from data_cleaner import DataCleaner
from model import Model
from preprocessor import Preprocessor
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor

In [17]:
util = Util()
cleaner = DataCleaner()
processor = Preprocessor()

In [3]:
# read our merged train and test dataset

# repo="https://github.com/Nathnael12/pharmaceutical-sales-pridiction.git"
repo="../"

# store=util.read_from_dvc("data/store.csv",repo,"store-v1",low_memory=False)
# complete_test=util.read_from_dvc("data/test.csv",repo,"test-v2",low_memory=False)

# complete_train=util.read_from_dvc("data/train.csv",repo,"train-v2",low_memory=False)

complete_train=pd.read_csv('../data/tst.csv')



In [4]:
complete_train = cleaner.fill_zeros(complete_train, ['PromoInterval'])
complete_train = cleaner.convert_to_string(complete_train, ['PromoInterval'])
sales_train = complete_train[complete_train['Open'] != 0 ]

In [9]:
complete_train2=complete_train.copy()

In [10]:

sales_train = complete_train2[complete_train2['Open'] != 0 ]

In [11]:
processed_csv = processor.prepare(sales_train)

In [12]:
# run this only for the first time
# processed_csv.to_csv('../data/processed_csv.csv',index=False)

In [13]:
random_forest_args = {
    'n_estimators': 100,
    'max_depth': 16,
    'max_features': 'sqrt',
    'n_jobs': 4
}

In [15]:
modeling = Model(processed_csv, RandomForestRegressor)

In [ ]:
model, result = modeling.train(random_forest_args, 'random_forest_regressor')

In [ ]:
modeling.prediction_graph(result)